In [2]:
import sys
sys.path.append("..")
from lib.util import add_income_group, add_race_eth, add_purpose_type
import pandas as pd
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas


In [3]:
# Add the race an ethnicity column, race_eth
add_race_eth(df)

dfs = df[['id','census_tract_number', 'applicant_ethnicity','applicant_race_1', 'race_eth']]
dfc = dfs.groupby(['census_tract_number', 'race_eth']).count()[['id']]

# Pivot to get the race_eth as columns
df_re = dfc.reset_index().pivot('census_tract_number', 'race_eth','id')
df_re.head()


race_eth,aian,as,b,h,pi,w
census_tract_number,,,,,,
0001.00,1,7,3,15,NaN,347
0002.01,NaN,7,NaN,12,2,189
0002.02,NaN,23,2,21,NaN,355
0003.00,2,15,3,18,1,282
0004.00,NaN,9,2,2,NaN,115


In [4]:
# Needs to be converted from a string, since some entries are not numbers
df.applicant_income = df.applicant_income.astype(float, raise_on_error=False)

# Add the 'income_group'
add_income_group(df)

In [5]:
dfi = df[['applicant_income','hud_median_family_income', 'tract_to_msa_md_income','income_group']]
dfi.income_group.value_counts()

low     207208
na       28465
mod       4278
mid       1557
high      1198
dtype: int64

In [6]:
msa = df[(df.msa_md  == 41740) & ( df.respondent_id == '0000480228' )].copy() # San Diego MSA / Bank of America
add_income_group(msa) # adds income_group
add_race_eth(msa) # adds race_eth
add_purpose_type(msa) # add purpose_type

In [46]:
a = msa.groupby(['purpose_type', 'action_type']).count()[['id']].unstack()
b = msa.groupby(['purpose_type', 'action_type']).sum()[['loan_amount']].unstack()
c = a.merge(b, left_index = True, right_index = True)
p_type_frame = c.sortlevel(1, axis=1)

p_type_frame['group'] = 'Purpose and Type'
p_type_frame.set_index('group', append=True, inplace=True)
p_type_frame = p_type_frame.reorder_levels(['group', 'purpose_type'])
p_type_frame

id loan_amount   id loan_amount  \
action_type                                1           1    2           2   
group            purpose_type                                               
Purpose and Type improvement              52       17532    1         546   
                 purchase-conventional   622      294449   43       31246   
                 purchase-govt           243       81031  NaN         NaN   
                 refinance              5439     1867375  214       71990   

                                          id loan_amount   id loan_amount  \
action_type                                3           3    4           4   
group            purpose_type                                               
Purpose and Type improvement              35       10083    6         738   
                 purchase-conventional   204       80438  115       61614   
                 purchase-govt           100       32482   48       16806   
                 refinance              1577      524407  601      175489   

                                         id loan_amount   id loan_amount  
action_type                               5           5    6           6  
group            purpose_type                                             
Purpose and Type improvement              6        2554    3         964  
                 purchase-conventional   13        4072   25       12195  
                 purchase-govt            5        1701  154       42166  
                 refinance              296      106825  144       37632

In [50]:
a = msa.groupby(['race_eth', 'action_type']).count()[['id']].unstack()
b = msa.groupby(['race_eth', 'action_type']).sum()[['loan_amount']].unstack()
c = a.merge(b, left_index = True, right_index = True)
race_eth_frame = c.sortlevel(1, axis=1)
race_eth_frame['group'] = 'Race Eth'
race_eth_frame.set_index('group', append=True, inplace=True)
race_eth_frame = race_eth_frame.reorder_levels(['group', 'race_eth'])
race_eth_frame

id loan_amount   id loan_amount    id loan_amount   id  \
action_type           1           1    2           2     3           3    4   
group    race_eth                                                             
Race Eth aian        22        6991    1         310     8        1679    5   
         as         654      213490   27        7890   203       58954   88   
         b          133       37938    1         175    36        9744   12   
         h          829      205775   23        5658   331       79187  136   
         pi          49       14324    3         966    12        2879    7   
         w         3721     1358351  154       64096  1032      378001  422   

                  loan_amount   id loan_amount   id loan_amount  
action_type                 4    5           5    6           6  
group    race_eth                                                
Race Eth aian            1292    1         227    3        1500  
         as             28644   36       15054   15        5366  
         b               3165   10        2870    9        2429  
         h              29441   66       15568   78       19344  
         pi              1379    4        1192    6        1662  
         w             139075  164       62328  175       50572

In [51]:
pd.concat([p_type_frame, race_eth_frame])

id loan_amount   id loan_amount  \
action_type                                1           1    2           2   
group            purpose_type                                               
Purpose and Type improvement              52       17532    1         546   
                 purchase-conventional   622      294449   43       31246   
                 purchase-govt           243       81031  NaN         NaN   
                 refinance              5439     1867375  214       71990   
Race Eth         aian                     22        6991    1         310   
                 as                      654      213490   27        7890   
                 b                       133       37938    1         175   
                 h                       829      205775   23        5658   
                 pi                       49       14324    3         966   
                 w                      3721     1358351  154       64096   

                                          id loan_amount   id loan_amount  \
action_type                                3           3    4           4   
group            purpose_type                                               
Purpose and Type improvement              35       10083    6         738   
                 purchase-conventional   204       80438  115       61614   
                 purchase-govt           100       32482   48       16806   
                 refinance              1577      524407  601      175489   
Race Eth         aian                      8        1679    5        1292   
                 as                      203       58954   88       28644   
                 b                        36        9744   12        3165   
                 h                       331       79187  136       29441   
                 pi                       12        2879    7        1379   
                 w                      1032      378001  422      139075   

                                         id loan_amount   id loan_amount  
action_type                               5           5    6           6  
group            purpose_type                                             
Purpose and Type improvement              6        2554    3         964  
                 purchase-conventional   13        4072   25       12195  
                 purchase-govt            5        1701  154       42166  
                 refinance              296      106825  144       37632  
Race Eth         aian                     1         227    3        1500  
                 as                      36       15054   15        5366  
                 b                       10        2870    9        2429  
                 h                       66       15568   78       19344  
                 pi                       4        1192    6        1662  
                 w                      164       62328  175       50572